# Imputacion de Datos Faltantes para la Precipitacion del Meta mediante modelacion KNN

Este Jupyter Notebook se enfoca en el desarrolo metodologico del trabajo de grado de los estudiantes Ronaldo Hernandez Mejia y Hector Freddy Velasco, en el cual se enfoca en la imputaciopn de datos faltnates para la variable de precipitacion en el departamento del Meta, Colombia. Con una gran volumen de datos de la informacion diaria de la precipitacion desde 1980 hasta el 2016, seleccionando un total de 56 Estaciones meteorologias del IDEAM.

La región del Meta es muy importante en este país por la diversidad del clima y su efecto sobre la agricultura y otros sectores económicos. La variabilidad de las precipitaciones juega un papel importante en el análisis del clima y la toma de decisiones relacionadas con  la gestión del agua y el clima. 

El objetivo principal es desarrolar un modelo basado en el algotimo K-Nearest Neighbors(KNN) para imputar los datos faltantes de precipitacion, utilizando un conjunto de datos que contiene mas de 700.000 observaciones. 

Dada la gran cantidad de datos se ha implementado una version que utiliza los cores del procesador para ejecutar el modelo en paralelo optimizando asi el tiempo de ejecucion, mejorando la eficiencia computacional y acelerar el proceso, tambien por este motivo el desarrolo en python tiene tiempos de respuesta cortos en el manejo de datos.



### instalacion de librerias

In [1]:
import subprocess
import sys
import importlib

def install_and_import(package):
    try:
        importlib.import_module(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        globals()[package] = importlib.import_module(package)


In [2]:
packages = ['pandas', 'IPython', 'pandasgui', 'numpy', 'matplotlib','joblib','sklearn']

for package in packages:
    install_and_import(package)


En caso de que no funcione instalacion manual:

In [3]:
pip install -U pip 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sklearn


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install wheel


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
#observar donde estoy en ruta 
os.getcwd()

'c:\\Users\\vhect\\Documents\\semestre 10\\TESIS\\Proyecto_mod_precipitacion\\Codigos de Python'

ruta ronaldo 

In [12]:
import os
current_path = os.getcwd()
print(f"Ruta actual del directorio de trabajo: {current_path}")
new_path = "F:\\TESIS\\Proyecto_mod_precipitacion"
os.chdir(new_path)
new_current_path = os.getcwd()
print(f"Nueva ruta del directorio de trabajo: {new_current_path}")

Ruta actual del directorio de trabajo: f:\TESIS\Proyecto_mod_precipitacion\Codigos de Python
Nueva ruta del directorio de trabajo: F:\TESIS\Proyecto_mod_precipitacion


ruta hector 


In [7]:
import os
current_path = os.getcwd()
print(f"Ruta actual del directorio de trabajo: {current_path}")
new_path = "c:\\Users\\vhect\\Documents\\semestre 10\\TESIS\\Proyecto_mod_precipitacion"
os.chdir(new_path)
new_current_path = os.getcwd()
print(f"Nueva ruta del directorio de trabajo: {new_current_path}")

Ruta actual del directorio de trabajo: c:\Users\vhect\Documents\semestre 10\TESIS\Proyecto_mod_precipitacion\Codigos de Python
Nueva ruta del directorio de trabajo: c:\Users\vhect\Documents\semestre 10\TESIS\Proyecto_mod_precipitacion


### Ejemplo sencillo del funcionamiento del modelo KNN

modelo KNN 
scikit-learn es una biblioteca de aprendizaje automático de código 
abierto que ofrece implementaciones eficientes de una amplia variedad
de algoritmos de aprendizaje, incluyendo KNN. Además, proporciona funciones
para la imputación de datos faltantes utilizando KNN y otras técnicas.

In [8]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer


In [ ]:

def impute_precipitation_for_station(station_data, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    station_data_imputed = imputer.fit_transform(station_data[['precipitation']])
    station_data['precipitation_imputed'] = station_data_imputed
    return station_data

# Crear datos de ejemplo
data = pd.DataFrame({
    'station_id': [1, 1, 2, 2, 3, 3],
    'precipitation': [1, np.nan, 3, 4, np.nan, 6]
})

stations = data['station_id'].unique()
imputed_data = pd.DataFrame()

for station in stations:
    station_data = data[data['station_id'] == station]
    station_data_imputed = impute_precipitation_for_station(station_data)
    imputed_data = imputed_data.append(station_data_imputed)


Presenta el Dataframe con las variables imputadas atravez de la metodologia KNN

In [10]:
import warnings
warnings.filterwarnings('ignore')

# Tu código
print(imputed_data)

   station_id  precipitation  precipitation_imputed
0           1            1.0                    1.0
1           1            NaN                    1.0
2           2            3.0                    3.0
3           2            4.0                    4.0
4           3            NaN                    6.0
5           3            6.0                    6.0


### Lectura de datos

In [11]:
import pandas as pd

In [12]:
data = pd.read_csv("Datos/datos_seleccionados_para_modelo_coordenadas.txt",sep=";")
data.head()

,Date,prec,ID,LAT,LON
0,1983-06-09,6.8,32010010,2.562833,-74.103083
1,1983-06-10,17.8,32010010,2.562833,-74.103083
2,1983-06-11,0.0,32010010,2.562833,-74.103083
3,1983-06-12,0.5,32010010,2.562833,-74.103083
4,1983-06-13,6.9,32010010,2.562833,-74.103083


Estructura de los datos

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656436 entries, 0 to 656435
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    656436 non-null  object 
 1   prec    592124 non-null  float64
 2   ID      656436 non-null  int64  
 3   LAT     656436 non-null  float64
 4   LON     656436 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 25.0+ MB


Aplicacion del modelo a los datos reales

In [14]:

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from joblib import Parallel, delayed

Funcion de imputacion 5 vecinos cercanos

In [15]:
def impute_precipitation_for_station(station_data, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    station_data_imputed = imputer.fit_transform(station_data[['prec']])
    station_data['prec_imputed'] = station_data_imputed
    return station_data



La función impute_precipitation_parallel ejecutará la imputación de datos de precipitación 
utilizando KNN Imputer en paralelo para todas las estaciones en tus datos. 
El parámetro n_jobs controla el número de procesos en paralelo. 
Si n_jobs=-1, se utilizarán todos los núcleos disponibles en tu computadora.

El parámetro n_jobs controla el número de procesos en paralelo. Si n_jobs=-1, 
se utilizarán todos los núcleos disponibles en tu computadora.

In [16]:
def impute_precipitation_parallel(data, n_jobs=5):
    stations = data['ID'].unique()
    results = Parallel(n_jobs=n_jobs)(
        delayed(impute_precipitation_for_station)(data[data['ID'] == station]) for station in stations
    )
    imputed_data = pd.concat(results, ignore_index=True)
    return imputed_data

data['Date'] = pd.to_datetime(data['Date']) 

Aplicacion de la funcion para los datos

In [17]:
imputed_data = impute_precipitation_parallel(data)
print(imputed_data)

             Date  prec        ID       LAT        LON  prec_imputed
0      1983-06-09   6.8  32010010  2.562833 -74.103083           6.8
1      1983-06-10  17.8  32010010  2.562833 -74.103083          17.8
2      1983-06-11   0.0  32010010  2.562833 -74.103083           0.0
3      1983-06-12   0.5  32010010  2.562833 -74.103083           0.5
4      1983-06-13   6.9  32010010  2.562833 -74.103083           6.9
...           ...   ...       ...       ...        ...           ...
656431 2014-07-14   0.1  35185010  4.260361 -72.564472           0.1
656432 2014-07-15   9.8  35185010  4.260361 -72.564472           9.8
656433 2014-07-16   0.0  35185010  4.260361 -72.564472           0.0
656434 2014-07-17   4.5  35185010  4.260361 -72.564472           4.5
656435 2014-07-18   0.0  35185010  4.260361 -72.564472           0.0

[656436 rows x 6 columns]


In [18]:
imputed_data.info()

imputed_data[['prec','prec_imputed']].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656436 entries, 0 to 656435
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          656436 non-null  datetime64[ns]
 1   prec          592124 non-null  float64       
 2   ID            656436 non-null  int64         
 3   LAT           656436 non-null  float64       
 4   LON           656436 non-null  float64       
 5   prec_imputed  656436 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 30.0 MB


,prec,prec_imputed
count,592124.000000,656436.000000
mean,8.915141,8.919318
std,17.865258,16.991514
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.500000
75%,10.000000,10.000000
max,438.800000,438.800000


## Bibliografia 

Little, R. J. A., & Rubin, D. B. (2002). Statistical Analysis with Missing Data (Vol. 333). John Wiley & Sons.